# リストを返さずにジェネレータを返すことを考える

In [32]:
# 空白があるインデックスを求める
def index_words(text):
  result = []
  if text:
    result.append(0)
  for index, letter in enumerate(text):
    print(f'{index} : {letter}')
    if letter == ' ':
      result.append(index + 1)
  return result

In [33]:
address = 'Four score and seven years ago'
result = index_words(address)
print(result[:10])

0 : F
1 : o
2 : u
3 : r
4 :  
5 : s
6 : c
7 : o
8 : r
9 : e
10 :  
11 : a
12 : n
13 : d
14 :  
15 : s
16 : e
17 : v
18 : e
19 : n
20 :  
21 : y
22 : e
23 : a
24 : r
25 : s
26 :  
27 : a
28 : g
29 : o
[0, 5, 11, 15, 21, 27]


In [34]:
# リストを返す index_words と違い、result.append や return がない

# ジェネレータ関数は、呼び出されると実際の作業をせず、イテレータを返す
# 無見込み関数 next が呼び出されるごとに、イテレータはジェネレータ式を次の yield 式に進める

# 結果リストに関する処理のすべてが省かれているので、はるかに読みやすい
def index_words_iter(text):
  if text:
    yield 0
  for index, letter in enumerate(text):
    if letter == ' ':
      yield index + 1

In [35]:
it = index_words_iter(address)
print(next(it))
print(next(it))

0
5


In [36]:
# ジェネレータ呼び出しで返されるイテレータは、組み込み関数listに渡して簡単にリストに変換可能
result = list(index_words_iter(address))
print(result[:10])

[0, 5, 11, 15, 21, 27]


In [2]:
# index_words は、返す前に、すべての結果をリストに格納する必要がある
# 入力が大量にあるときに、メモリを食いつぶしてしまう

# 対象にジェネレータは一定のメモリしか必要としないので、どんな長さの入力にも容易に対応
def index_file(handle):
  offset = 0
  for line in handle: # ファイルを1行ずつ読む設計になっている ---ファイルが 10GB あろうと、メモリ上に存在するのは常に「今の1行」だけ ---
    print(f'line : {line}') # 1行ずつなのがわかる
    if line:
      yield offset # 行の開始位置を記録
    for letter in line:
      offset += 1
      if letter == ' ':
        yield offset # 空白の直後の位置も記録

In [6]:
from itertools import islice

with open('address.txt', 'r') as f:
  it = index_file(f)
  result = islice(it, 0 , 30)
  print(list(result))

line : Four score and seven years ago our fathers brought forth

line : on this continent a new nation, conceived in liberty, and

line : dedicated to the proposition that all men are created equal.

[0, 5, 11, 15, 21, 27, 31, 35, 43, 51, 57, 60, 65, 75, 77, 81, 89, 99, 102, 111, 115, 125, 128, 132, 144, 149, 153, 157, 161, 169]


In [5]:
with open('address.txt', 'r') as f:
  it = index_file(f)
  result = islice(it, 0 , 10)
  result_next = islice(it, 0 , 10)
  result_next2 = islice(it, 0 , 10)
  print(list(result))
  print(list(result_next))
  print(list(result_next2))

line : Four score and seven years ago our fathers brought forth

[0, 5, 11, 15, 21, 27, 31, 35, 43, 51]
line : on this continent a new nation, conceived in liberty, and

[57, 60, 65, 75, 77, 81, 89, 99, 102, 111]
line : dedicated to the proposition that all men are created equal.

[115, 125, 128, 132, 144, 149, 153, 157, 161, 169]


このようなジェネレータを定義するときは、返されるイテレータがステートフルで再利用できないことを呼び出し元が認識すべきである

In [ ]:
def index_file_list_v(handle):
  offsets = []  # すべての結果をこのリストにためる
  offset = 0
  for line in handle:
    print(f'line : {line}') # すべて読み込む
    if line:
      offsets.append(offset)  # yieldの代わりにリストに追加
    for letter in line:
      offset += 1
      if letter == ' ':
        offsets.append(offset)  # これもリストに追加
  return offsets  # 最後にすべて返す

In [47]:
with open('address.txt', 'r') as f:
  offsets = index_file_list_v(f)
  print(f'print offsets : {offsets}')
  print(offsets[:10])
#   for offset in offsets:
#     print(offset)  # リストを全部メモリに保持してから使う

line : Four score and seven years ago our fathers brought forth

line : on this continent a new nation, conceived in liberty, and

line : dedicated to the proposition that all men are created equal.

line : Now we are engaged in a great civil war, testing

line : whether that nation, or any nation so conceived and so

line : dedicated, can long endure. We are met on a great battlefield of that war. We have come to dedicate a portion of

line : that field as a final resting-place for those who here gave

line : their lives that this nation might live. It is altogether

line : fitting and proper that we should do this.
print offsets : [0, 5, 11, 15, 21, 27, 31, 35, 43, 51, 57, 60, 65, 75, 77, 81, 89, 99, 102, 111, 115, 125, 128, 132, 144, 149, 153, 157, 161, 169, 176, 180, 183, 187, 195, 198, 200, 206, 212, 217, 225, 233, 238, 246, 249, 253, 260, 263, 273, 277, 280, 291, 295, 300, 308, 311, 315, 319, 322, 324, 330, 342, 345, 350, 355, 358, 363, 368, 371, 380, 382, 390, 393, 398, 404, 407

## まとめ

* リスト版: 全部のオフセットを一気にメモリにためるので、メモリを爆食いする
* ジェネレータ版: 1つずつ処理してすぐ捨てるから、省メモリ

## 【実験コード】リスト vs ジェネレータのメモリ使用量比較

In [48]:
import os
import tracemalloc  # メモリ使用量を計測する標準ライブラリ
from itertools import islice

# 大きめのファイルを仮想的に作る関数
def create_large_test_file(filename, num_lines=100000, line_length=100):
    with open(filename, 'w') as f:
        for _ in range(num_lines):
            line = 'word ' * (line_length // 5) + '\n'  # 1行あたり約100文字
            f.write(line)

# ジェネレータ版
def index_file_generator(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

# リスト版
def index_file_list(handle):
    offsets = []
    offset = 0
    for line in handle:
        if line:
            offsets.append(offset)
        for letter in line:
            offset += 1
            if letter == ' ':
                offsets.append(offset)
    return offsets

# メモリ計測用関数
def measure_memory(func, *args, n=10):
    tracemalloc.start()
    result = func(*args)
    snapshot = tracemalloc.take_snapshot()
    tracemalloc.stop()
    # 上位n個のメモリ使用情報を取得
    top_stats = snapshot.statistics('lineno')
    total = sum(stat.size for stat in top_stats)
    return total, result

# 実験スタート
if __name__ == "__main__":
    filename = 'test_large_file.txt'
    
    if not os.path.exists(filename):
        print("テストファイル作成中...")
        create_large_test_file(filename)

    print("==== メモリ比較スタート ====")

    with open(filename, 'r') as f:
        print("\n--- ジェネレータ版 ---")
        memory_gen, gen_obj = measure_memory(index_file_generator, f)
        small_sample = list(islice(gen_obj, 10))  # 少しだけ取り出して確認
        print(f"メモリ使用量: {memory_gen / 1024:.2f} KB")
        print(f"最初の10個: {small_sample}")

    with open(filename, 'r') as f:
        print("\n--- リスト版 ---")
        memory_list, list_result = measure_memory(index_file_list, f)
        print(f"メモリ使用量: {memory_list / 1024:.2f} KB")
        print(f"最初の10個: {list_result[:10]}")

    print("\n==== メモリ比較終了 ====")

テストファイル作成中...
==== メモリ比較スタート ====

--- ジェネレータ版 ---
メモリ使用量: 0.23 KB
最初の10個: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

--- リスト版 ---
メモリ使用量: 82351.53 KB
最初の10個: [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

==== メモリ比較終了 ====


#### 【このコードの流れ】
テスト用に「10万行 × 1行100文字くらい」のちょっと大きめのファイルを自動生成する。

1. ジェネレータ版でoffsetを取り出し、メモリ使用量を計測

2. リスト版でoffsetをリストに溜めて、メモリ使用量を計測

3. 両者のメモリ量と、最初の10個のデータを表示！